In [1]:
import numpy as np
from skimage.transform import resize
from keras.models import Sequential
from keras.layers import Dense, Flatten, CuDNNLSTM, RepeatVector, TimeDistributed, ConvLSTM2D
import pylidc as pl
from unet import UNet

/Users/amandazhang/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
image_dims = (256, 256, 1)
num_slices = 20

In [43]:
model = Sequential()
#model.add(TimeDistributed(UNet(image_dims, start_ch=32, batchnorm=True), input_shape=(num_slices,) + image_dims))
model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3), 
                     input_shape=(num_slices, 256, 256, 1),
                     padding='same', return_sequences=True))
#model.add(TimeDistributed(Flatten()))
#model.add(RepeatVector(num_slices))
#model.add(CuDNNLSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [3]:
scans = pl.query(pl.Scan)
n = scans.count()

In [9]:
np.array(get_data(0)).shape

ValueError: could not broadcast input array from shape (20,256,256,1) into shape (1,20)

In [4]:
def get_annotated_slices(ann):
    zs = ann.bbox()[-1]
    return zs.start, zs.stop

In [5]:
def get_data(i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([[resize(im.pixel_array, (256, 256), mode='constant') for im in images]])
    nodule_zs = set.union(*(set(range(*get_annotated_slices(ann))) for ann in scan.annotations))
    y = np.array([[i in nodule_zs for i in range(n)]])
    start = int((n - num_slices) / 2)
    X = X[:, start:start + num_slices]
    y = y[:, start:start + num_slices]
    return np.expand_dims(X, axis=-1), np.expand_dims(y, axis=-1)

In [12]:
def datagen():
    i = np.random.randint(n)
    yield get_data(i)

In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_3 (ConvLSTM2D)  (None, 20, 256, 256, 40)  59200     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 256, 256, 1)   41        
Total params: 59,241
Trainable params: 59,241
Non-trainable params: 0
_________________________________________________________________


In [42]:
num_epochs = 2
for epoch in range(num_epochs):
    for i in range(n):
        data = get_data(i)
        print(data[0].shape)
        model.fit(data[0], data[1].astype('int32'), verbose=True)

(1, 20, 256, 256, 1)


ValueError: Error when checking target: expected time_distributed_3 to have 5 dimensions, but got array with shape (1, 20, 1)

In [38]:
data[1].astype('int32')

array([[[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0]]], dtype=int32)

In [25]:
helo = get_data(i)[0]

In [26]:
helo

array([[[[[-0.06248569],
          [-0.06248569],
          [-0.06248569],
          ...,
          [-0.06248569],
          [-0.06248569],
          [-0.06248569]],

         [[-0.06248569],
          [-0.06248569],
          [-0.06248569],
          ...,
          [-0.06248569],
          [-0.06248569],
          [-0.06248569]],

         [[-0.06248569],
          [-0.06248569],
          [-0.06248569],
          ...,
          [-0.06248569],
          [-0.06248569],
          [-0.06248569]],

         ...,

         [[-0.06248569],
          [-0.06248569],
          [-0.06248569],
          ...,
          [-0.06248569],
          [-0.06248569],
          [-0.06248569]],

         [[-0.06248569],
          [-0.06248569],
          [-0.06248569],
          ...,
          [-0.06248569],
          [-0.06248569],
          [-0.06248569]],

         [[-0.06248569],
          [-0.06248569],
          [-0.06248569],
          ...,
          [-0.06248569],
          [-0.06248569],
          

In [9]:
d = datagen()

In [10]:
dir(d)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [7]:
X, y = get_data(5)

In [8]:
model.fit(X, y, epochs=1)

Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[20,128,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: time_distributed_1/conv2d_15/convolution = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](time_distributed_1/concatenate_2/concat, conv2d_15/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics/acc/Mean_1/_1661 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_9422_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated

In [48]:
delt = 1.
delxp = 1.
delx = 100000.
c = 299792458.

print(delt**2 + delxp**2/c**2)
print(-2*delx*delt)
print(delx**2 - delxp**2)

1.0
-200000.0
9999999999.0


In [52]:
v = 99999.
v = 101000.

print( (delt - (v*delx / c**2))/(np.sqrt(1-v**2/c**2)) )

0.9999999443730591


In [50]:
v**2/c**2

1.1126278031637625e-07